# SQL Basics

## Introduction

This notebook is a short introduction to SQL. It is based on the [DuckDB](https://duckdb.org/) database engine.

## Datasets

The following datasets are used in this notebook. You don't need to download them, they can be accessed directly from the notebook.

- [cities.csv](https://storage.googleapis.com/qm2/CASA0025/cities.csv)
- [countries.csv](https://storage.googleapis.com/qm2/CASA0025/countries.csv)

## References

- [W3Schools SQL Tutorial](https://www.w3schools.com/sql)
- [DuckDB SQL Introduction](https://duckdb.org/docs/sql/introduction.html)

## Installation

Uncomment the following cell to install the required packages.

In [1]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 kB 14.1 MB/s eta 0:00:00


## Library Import and Configuration

In [2]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

Set configurations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.

In [3]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Connecting to DuckDB

Connect jupysql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

In [4]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db

 If your SQL query is one line only, you may use the `%sql` magic command. For multi-line SQL query, you may use the `%%sql` magic command.

## Install extensions


Check available DuckDB extensions.

In [5]:
%%sql

SELECT * FROM duckdb_extensions();

,extension_name,loaded,installed,install_path,description,aliases,extension_version,install_mode,installed_from
0,autocomplete,False,False,,Adds support for autocomplete in the shell,[],,NOT_INSTALLED,
1,aws,False,False,,Provides features that depend on the AWS SDK,[],,NOT_INSTALLED,
2,azure,False,False,,Adds a filesystem abstraction for Azure blob s...,[],,NOT_INSTALLED,
3,core_functions,True,True,(BUILT-IN),Core function library,[],,STATICALLY_LINKED,
4,delta,False,False,,Adds support for Delta Lake,[],,NOT_INSTALLED,
5,ducklake,False,False,,"Adds support for DuckLake, SQL as a Lakehouse ...",[],,NOT_INSTALLED,
6,encodings,False,False,,All unicode encodings to UTF-8,[],,NOT_INSTALLED,
7,excel,False,False,,Adds support for Excel-like format strings,[],,NOT_INSTALLED,
8,fts,False,False,,Adds support for Full-Text Search Indexes,[],,NOT_INSTALLED,
9,httpfs,False,False,,Adds support for reading and writing files ove...,"[http, https, s3]",,NOT_INSTALLED,


DuckDB's [httpfs extension](https://duckdb.org/docs/extensions/httpfs) allows parquet and csv files to be queried remotely over http. This is useful for querying large datasets without having to download them locally. Let's install the extension and load the extension.

In [6]:
%%sql

INSTALL httpfs;
LOAD httpfs;

,Success


from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_0.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

## Read CSV

Use the `httpfs` extension to read the `cities.csv` file from the web.

In [7]:
%%sql

SELECT * FROM 'https://storage.googleapis.com/qm2/CASA0025/cities.csv';

,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.58330,32.53330,75000
1,2,Fort Portal,UGA,0.67100,30.27500,42670
2,3,Potenza,ITA,40.64200,15.79900,69060
3,4,Campobasso,ITA,41.56300,14.65600,50762
4,5,Aosta,ITA,45.73700,7.31500,34062
...,...,...,...,...,...,...
1244,1245,Rio de Janeiro,BRA,-22.92502,-43.22502,11748000
1245,1246,Sao Paulo,BRA,-23.55868,-46.62502,18845000
1246,1247,Sydney,AUS,-33.92001,151.18518,4630000
1247,1248,Singapore,SGP,1.29303,103.85582,5183700


In [8]:
%%sql

SELECT * FROM 'https://storage.googleapis.com/qm2/CASA0025/countries.csv';

,id,Country,Alpha2_code,Alpha3_code,Numeric_code,Latitude,Longitude
0,1,Afghanistan,AF,AFG,4,33.0000,65.0
1,2,Albania,AL,ALB,8,41.0000,20.0
2,3,Algeria,DZ,DZA,12,28.0000,3.0
3,4,American Samoa,AS,ASM,16,-14.3333,-170.0
4,5,Andorra,AD,AND,20,42.5000,1.6
...,...,...,...,...,...,...,...
238,239,Wallis and Futuna,WF,WLF,876,-13.3000,-176.2
239,240,Western Sahara,EH,ESH,732,24.5000,-13.0
240,241,Yemen,YE,YEM,887,15.0000,48.0
241,242,Zambia,ZM,ZMB,894,-15.0000,30.0


## Create Table

Create a table named `cities` from the `cities.csv` file.

In [9]:
%%sql

CREATE TABLE cities AS SELECT * FROM 'https://storage.googleapis.com/qm2/CASA0025/cities.csv';

,Success


Create a table named `countries` from the `countries.csv` file.

In [10]:
%%sql

CREATE TABLE countries AS SELECT * FROM 'https://storage.googleapis.com/qm2/CASA0025/countries.csv';

,Success


Display the table content in the database.

In [11]:
%%sql

FROM cities;

,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.58330,32.53330,75000
1,2,Fort Portal,UGA,0.67100,30.27500,42670
2,3,Potenza,ITA,40.64200,15.79900,69060
3,4,Campobasso,ITA,41.56300,14.65600,50762
4,5,Aosta,ITA,45.73700,7.31500,34062
...,...,...,...,...,...,...
1244,1245,Rio de Janeiro,BRA,-22.92502,-43.22502,11748000
1245,1246,Sao Paulo,BRA,-23.55868,-46.62502,18845000
1246,1247,Sydney,AUS,-33.92001,151.18518,4630000
1247,1248,Singapore,SGP,1.29303,103.85582,5183700


In [ ]:
%%sql

FROM countries;

## The SQL SELECT statement

The `SELECT` statement is used to select data from a database. Use either `SELECT *` to select all columns, or `SELECT column1, column2, ...` to select specific columns.

`SELECT * FROM cities` is the same as `FROM cities`.

In [12]:
%%sql

SELECT * FROM cities;

,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.58330,32.53330,75000
1,2,Fort Portal,UGA,0.67100,30.27500,42670
2,3,Potenza,ITA,40.64200,15.79900,69060
3,4,Campobasso,ITA,41.56300,14.65600,50762
4,5,Aosta,ITA,45.73700,7.31500,34062
...,...,...,...,...,...,...
1244,1245,Rio de Janeiro,BRA,-22.92502,-43.22502,11748000
1245,1246,Sao Paulo,BRA,-23.55868,-46.62502,18845000
1246,1247,Sydney,AUS,-33.92001,151.18518,4630000
1247,1248,Singapore,SGP,1.29303,103.85582,5183700


To limit the number of rows returned, use the `LIMIT` keyword. For example, `SELECT * FROM cities LIMIT 10` will return only the first 10 rows.

In [13]:
%%sql

SELECT * FROM cities LIMIT 10;

,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.58330,32.53330,75000
1,2,Fort Portal,UGA,0.67100,30.27500,42670
2,3,Potenza,ITA,40.64200,15.79900,69060
3,4,Campobasso,ITA,41.56300,14.65600,50762
4,5,Aosta,ITA,45.73700,7.31500,34062
5,6,Mariehamn,ALD,60.09700,19.94900,10682
6,7,Ramallah,PSE,31.90294,35.20621,24599
7,8,Vatican City,VAT,41.90001,12.44781,832
8,9,Poitier,FRA,46.58329,0.33328,85960
9,10,Clermont-Ferrand,FRA,45.77998,3.08001,233050


Select a subset of columns from the `cities` table and display the first 10 rows.

In [14]:
%%sql

SELECT name, country FROM cities LIMIT 10;

,name,country
0,Bombo,UGA
1,Fort Portal,UGA
2,Potenza,ITA
3,Campobasso,ITA
4,Aosta,ITA
5,Mariehamn,ALD
6,Ramallah,PSE
7,Vatican City,VAT
8,Poitier,FRA
9,Clermont-Ferrand,FRA


To select distinct values, use the `DISTINCT` keyword. For example, `SELECT DISTINCT country FROM cities` will return only the distinct values of the `country` column.

In [18]:
%%sql

SELECT DISTINCT country FROM cities LIMIT 10;

,country
0,ALD
1,FRA
2,ATA
3,ZMB
4,JPN
5,SRB
6,BIH
7,GBR
8,SMR
9,SWZ


To count the number of rows returned, use the `COUNT(*)` function. For example, `SELECT COUNT(*) FROM cities` will return the number of rows in the `cities` table.

In [19]:
%%sql

SELECT COUNT(*) FROM cities;

,count_star()
0,1249


To count the number of distinct values, use the `COUNT(DISTINCT column)` function. For example, `SELECT COUNT(DISTINCT country) FROM cities` will return the number of distinct values in the `country` column.

In [20]:
%%sql

SELECT COUNT(DISTINCT country) FROM cities;

,count(DISTINCT country)
0,200


To calculate the maximum value, use the `MAX(column)` function. For example, `SELECT MAX(population) FROM cities` will return the maximum value in the `population` column.

In [ ]:
%%sql

SELECT MAX(population) FROM cities;

To calculate the total value, use the `SUM(column)` function. For example, `SELECT SUM(population) FROM cities` will return the total value in the `population` column.

In [ ]:
%%sql

SELECT SUM(population) FROM cities;

To calculate the average value, use the `AVG(column)` function. For example, `SELECT AVG(population) FROM cities` will return the average value in the `population` column.

In [ ]:
%%sql

SELECT AVG(population) FROM cities;

To order the results, use the `ORDER BY column` clause. For example, `SELECT * FROM cities ORDER BY country` will return the rows ordered by the `country` column alphabetically.

In [ ]:
%%sql

SELECT * FROM cities ORDER BY country LIMIT 10;

To order the results in descending order, use the `ORDER BY column DESC` clause. For example, `SELECT * FROM cities ORDER BY country ASC, population DESC` will return the rows ordered by the `country` column alphabetical order and then by the `population` column in descending order.

In [ ]:
%%sql

SELECT * FROM cities ORDER BY country ASC, population DESC LIMIT 10;

## The WHERE Clause

The `WHERE` clause is used to filter records. The `WHERE` clause is used to extract only those records that fulfill a specified condition.

In [ ]:
%%sql

SELECT * FROM cities WHERE country='USA'

You can use boolean operators such as `AND`, `OR`, `NOT` to filter records. For example, `SELECT * FROM cities WHERE country='USA' OR country='CAN'` will return the rows where the `country` column is either `USA` or `CAN`.

In [ ]:
%%sql

SELECT * FROM cities WHERE country='USA' OR country='CAN';

To select US cities with a population greater than 1 million, use the following query: `SELECT * FROM cities WHERE country='USA' AND population > 1000000`.

In [ ]:
%%sql

SELECT * FROM cities WHERE country='USA' AND population>1000000;

To select cities with the country name starting with the letter `U`, use the following query: `SELECT * FROM cities WHERE country LIKE 'U%'`.

In [ ]:
%%sql

SELECT * FROM cities WHERE country LIKE 'U%';

To select cities with the country name ending with the letter `A`, use the following query: `SELECT * FROM cities WHERE country LIKE '%A'`.

In [ ]:
%%sql

SELECT * FROM cities WHERE country LIKE '%A';

To select cities with the country name containing the letter `S` in the middle, use the following query: `SELECT * FROM cities WHERE country LIKE '_S_'`.

In [ ]:
%%sql

SELECT * FROM cities WHERE country LIKE '_S_';

To select cities from a list of countries, use the `IN` operator. For example, `SELECT * FROM cities WHERE country IN ('USA', 'CAN')` will return the rows where the `country` column is either `USA` or `CAN`.

In [ ]:
%%sql

SELECT * FROM cities WHERE country IN ('USA', 'CAN');

To select cities with a population between 1 and 10 million, use the following query: `SELECT * FROM cities WHERE population BETWEEN 1000000 AND 10000000`.

In [ ]:
%%sql

SELECT * FROM cities WHERE population BETWEEN 1000000 AND 10000000;

## SQL Joins

Reference: https://www.w3schools.com/sql/sql_join.asp

Here are the different types of the JOINs in SQL:

- `(INNER) JOIN`: Returns records that have matching values in both tables
- `LEFT (OUTER) JOIN`: Returns all records from the left table, and the matched records from the right table
- `RIGHT (OUTER) JOIN`: Returns all records from the right table, and the matched records from the left table
- `FULL (OUTER) JOIN`: Returns all records when there is a match in either left or right table

![](https://i.imgur.com/mITYzuS.png)

We have two sample tables: `cities` and `countries`.

There are 1,249 cities in the `cities` table and 243 countries in the `countries` table.

In [ ]:
%%sql

SELECT COUNT(*) FROM cities;

In [ ]:
%%sql

SELECT * FROM cities LIMIT 10;

In [ ]:
%%sql

SELECT COUNT(*) FROM countries;

In [ ]:
%%sql

SELECT * FROM countries LIMIT 10;

### SQL Inner Join

The `INNER JOIN` keyword selects records that have matching values in both tables. In the example, we join the `cities` table with the `countries` table using the `country` column in the `cities` table and the `Alpha3_code` column in the `countries` table. The result contains 1,244 rows, indicating that there are 5 cities that do not have a matching country.

In [ ]:
%%sql

SELECT * FROM cities INNER JOIN countries ON cities.country = countries."Alpha3_code";

Only select the `city` and `country` columns from the `cities` table and the `Country` column from the `countries` table.

In [ ]:
%%sql

SELECT name, cities."country", countries."Country" FROM cities INNER JOIN countries ON cities.country = countries."Alpha3_code";

### SQL Left Join

The `LEFT JOIN` keyword returns all records from the left table (`cities`), and the matched records from the right table (`countries`). The result contains 1,249 rows, the same number of rows as the `cities` table.

In [ ]:
%%sql

SELECT * FROM cities LEFT JOIN countries ON cities.country = countries."Alpha3_code";

### SQL Right Join

The `RIGHT JOIN` keyword returns all records from the right table (`countries`), and the matched records from the left table (`cities`). The result contains 1,291 rows.

In [ ]:
%%sql

SELECT * FROM cities RIGHT JOIN countries ON cities.country = countries."Alpha3_code";

### SQL Full Join

The `FULL JOIN` keyword returns all records when there is a match in either left or right table. The result contains 1,296 rows.

In [ ]:
%%sql

SELECT * FROM cities FULL JOIN countries ON cities.country = countries."Alpha3_code";

### SQL Union

The `UNION` operator is used to combine the result-set of two or more `SELECT` statements.

In [ ]:
%%sql

SELECT country FROM cities
UNION
SELECT "Alpha3_code" FROM countries;

## Aggregation

### Group By

The `GROUP BY` statement groups rows that have the same values into summary rows, like "find the number of cities in each country".

The `GROUP BY` statement is often used with aggregate functions (`COUNT`, `MAX`, `MIN`, `SUM`, `AVG`) to group the result-set by one or more columns.

In [ ]:
%%sql

SELECT COUNT(name), country
FROM cities
GROUP BY country
ORDER BY COUNT(name) DESC;

In [ ]:
%%sql

SELECT countries."Country", COUNT(name)
FROM cities
LEFT JOIN countries ON cities.country = countries."Alpha3_code"
GROUP BY countries."Country"
ORDER BY COUNT(name) DESC;

### Having

The `HAVING` clause was added to SQL because the `WHERE` keyword could not be used with aggregate functions.

For example, to select countries with more than 40 cities:

In [ ]:
%%sql

SELECT COUNT(name), country
FROM cities
GROUP BY country
HAVING COUNT(name) > 40
ORDER BY COUNT(name) DESC;

In [ ]:
%%sql

SELECT countries."Country", COUNT(name)
FROM cities
LEFT JOIN countries ON cities.country = countries."Alpha3_code"
GROUP BY countries."Country"
HAVING COUNT(name) > 40
ORDER BY COUNT(name) DESC;

## Conditional statements

The `CASE` statement goes through conditions and returns a value when the first condition is met (like an `IF-THEN-ELSE` statement). So, once a condition is true, it will stop reading and return the result. If no conditions are true, it returns the value in the `ELSE` clause.

For example, to divide cities into 3 groups based on their population:

In [ ]:
%%sql

SELECT name, population,
CASE
    WHEN population > 10000000 THEN 'Megacity'
    WHEN population > 1000000 THEN 'Large city'
    ELSE 'Small city'
END AS category
FROM cities;

## Saving results

You can save the results of a query to a new table using the `CREATE TABLE AS` statement.

In [ ]:
%%sql

CREATE TABLE cities2 AS SELECT * FROM cities;

Show the new table content.

In [ ]:
%%sql

FROM cities2;

Use the `DROP TABLE` statement to delete the table.

In [ ]:
%%sql

DROP TABLE IF EXISTS cities_usa;
CREATE TABLE cities_usa AS (SELECT * FROM cities WHERE country = 'USA');

In [ ]:
%%sql

FROM cities_usa;

Use the `INSERT INTO` statement to insert rows into a table.

In [ ]:
%%sql

INSERT INTO cities_usa (SELECT * FROM cities WHERE country = 'CAN');

## SQL Comments

Comments are used to explain sections of SQL statements, or to prevent execution of SQL statements.


### Single line comMents

Single line comments start with --.

Any text between -- and the end of the line will be ignored (will not be executed).

The following example uses a single-line comment as an explanation:

In [ ]:
%%sql

SELECT * FROM cities LIMIT 10 -- This is a comment;

### Multi-line comments

Multi-line comments start with `/*` and end with `*/`.

Any text between `/*` and `*/` will be ignored.

The following example uses a multi-line comment as an explanation:

In [ ]:
%%sql

SELECT COUNT(name), country
FROM cities
/*
 * Adding Group by
 * Adding Order by
 */
GROUP BY country
ORDER BY COUNT(name) DESC
LIMIT 10;